In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url ="https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)



soup = BeautifulSoup(webpage_content, 'html.parser')
table = soup.find('table')  # Adjust based on actual HTML structure
rows = table.find_all('tr')

data = []
for row in rows:
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

df = pd.DataFrame(data, columns=['Date', 'Day of Week', 'Event'])

# Add the year to the date if missing
df['Date'] = df['Date'] + ' 2021'

# Convert to datetime, coercing errors to NaT
df['Date'] = pd.to_datetime(df['Date'], format='%B %d %Y', errors='coerce')

# Drop rows with invalid dates (NaT)
df = df.dropna(subset=['Date'])

# Set the Date column as the index
df.set_index('Date', inplace=True)

# Rename columns
df.rename(columns={'Day of Week': 'dow', 'Event': 'text'}, inplace=True)

# Clean the 'text' column
df['text'] = (
    df['text']
    .str.replace('\n', ' ')  # Replace newlines with a space
    .str.replace('\t', ' ')  # Replace tabs with a space
    .str.strip()  # Remove leading/trailing spaces
    .str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with a single space
)

print(df)